# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

weatherpy_csv = "output_data_file"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
vacationpy_df = pd.read_csv(weatherpy_csv)
vacationpy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hervey bay,-25.2986,152.8535,73.00,63,52,8.01,AU,1619495977
1,belmonte,-15.8631,-38.8828,75.20,78,20,5.75,BR,1619495978
2,punta arenas,-53.1500,-70.9167,44.60,93,90,5.75,CL,1619495978
3,geraldton,-28.7667,114.6000,87.80,20,20,3.44,AU,1619495979
4,oranjemund,-28.5500,16.4333,59.04,86,79,5.12,NaN,1619495979


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
locations = vacationpy_df[["Lat", "Lng"]]
hum = vacationpy_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
### Create new DataFrame fitting weather criteria
# * Narrow down the cities to fit weather conditions.
# * Drop any rows will null values.

In [9]:
vacationpy_weather_crit = vacationpy_df[vacationpy_df['Max Temp'] < 80] 
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]
vacationpy_weather_crit

<ipython-input-9-1ea31caf26ac>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
<ipython-input-9-1ea31caf26ac>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
<ipython-input-9-1ea31caf26ac>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
195,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084
250,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107
302,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128
421,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177
423,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007
425,dubbo,-32.2500,148.6167,75.2,29,0,2.30,AU,1619496178
570,silvania,-16.6589,-48.6081,73.4,73,0,4.61,BR,1619496237


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
for vacationpy_weather_crit

# hotel_df = vacationpy_weather_crit['Hotel Name']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
